1. IV/RV ratio for different vol estimates
2. If the ratio is more than xth percentile, take the short ATM straddle trade
3. Adjustment
4. Change expiry date if the edge persists
5. Close, if no edge is there
6. Repeat

In [ ]:
import pandas as pd
import pandasql as ps
from pandasql import *
import plotly
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import glob
import warnings
from datetime import datetime
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import math
import time
from functools import reduce

warnings.simplefilter("ignore")

# For the widget

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

date = 'Apr 28'
vol_date = '2023-04-28'

valid_expiry = [
    "2010-11-25", "2010-12-30", "2011-01-27", "2011-02-24", "2011-03-31", "2011-04-28", 
    "2011-05-26", "2011-06-30", "2011-07-28", "2011-08-25", "2011-09-29", "2011-10-25", 
    "2011-11-24", "2011-12-29", "2012-01-25", "2012-02-23", "2012-03-29", "2012-04-26", 
    "2012-05-31", "2012-06-28", "2012-07-26", "2012-08-30", "2012-09-27", "2012-10-25", 
    "2012-11-29", "2012-12-27", "2013-01-31", "2013-02-28", "2013-03-28", "2013-04-25", 
    "2013-05-30", "2013-06-27", "2013-07-25", "2013-08-29", "2013-09-26", "2013-10-31", 
    "2013-11-28", "2013-12-26", "2014-01-30", "2014-02-26", "2014-02-27", "2014-03-27", 
    "2014-04-24", "2014-05-29", "2014-06-26", "2014-07-31", "2014-08-28", "2014-09-25", 
    "2014-10-30", "2014-11-27", "2014-12-24", "2015-01-29", "2015-02-26", "2015-03-26", 
    "2015-04-30", "2015-05-28", "2015-06-25", "2015-07-30", "2015-08-27", "2015-09-24", 
    "2015-10-29", "2015-11-26", "2015-12-31", "2016-01-28", "2016-02-25", "2016-03-31", 
    "2016-04-28", "2016-05-26", "2016-06-30", "2016-07-28", "2016-08-25", "2016-09-29", 
    "2016-10-27", "2016-11-24", "2016-12-29", "2017-01-25", "2017-02-23", "2017-03-30", 
    "2017-04-27", "2017-05-25", "2017-06-29", "2017-07-27", "2017-08-31", "2017-09-28", 
    "2017-10-26", "2017-11-30", "2017-12-28", "2018-01-25", "2018-02-22", "2018-03-28", 
    "2018-03-29", "2018-04-26", "2018-05-31", "2018-06-28", "2018-07-26", "2018-08-30", 
    "2018-09-27", "2018-10-25", "2018-11-29", "2018-12-27", "2019-01-31", "2019-02-28", 
    "2019-03-28", "2019-04-25", "2019-05-30", "2019-06-27", "2019-07-25", "2019-08-29", 
    "2019-09-26", "2019-10-31", "2019-11-28", "2019-12-26", "2020-01-30", "2020-02-27", 
    "2020-03-26", "2020-04-30", "2020-05-28", "2020-06-25", "2020-07-30", "2020-08-27", 
    "2020-09-24", "2020-10-29", "2020-11-26", "2020-12-31", "2021-01-28", "2021-02-25", 
    "2021-03-25", "2021-04-29", "2021-05-27", "2021-06-24", "2021-07-29", "2021-08-26", 
    "2021-09-30", "2021-10-28", "2021-11-25", "2021-12-30", "2022-01-27", "2022-02-24", 
    "2022-03-31", "2022-04-28", "2022-05-26", "2022-06-30", "2022-07-28", "2022-08-25",
    "2022-09-29", "2022-10-27", "2022-11-24", "2022-12-29", "2023-01-25", "2023-02-23", 
    "2023-03-29", "2023-04-27", "2023-05-25", "2023-06-29"
]

In [ ]:
# Get NIFTY options data

path = '/Users/vamsikrishnasb/My Drive/Financial Analysis/backtesting/bhav/options_02_split/NIFTY/Apr 28/Merged/iv_greeks.csv'
df = pd.read_csv(path)

In [ ]:
# ATM Option prices data for 'days to expiry' that are between 15 and 75
# (e.g.) If on a particular date, options from 3 expiries are trading, the code will pick
# only the ATM strike for each of the expiry.
# Additionally, the next closest monthly expiry has been chosen.
dte_14_42_df = df[
    (df['days_to_expiry'].between(14, 42))
    & (df['expiry'].isin(valid_expiry))
#     & ((df['strike'] % 100) == 0)
].reset_index().drop('index', axis=1)
dte_14_42_df['date_expiry'] = dte_14_42_df['date'] + '_' + dte_14_42_df['expiry']
dte_14_42_df['moneyness_abs'] = dte_14_42_df['moneyness_close'].abs()
temp_grouped = dte_14_42_df.groupby('date_expiry')['moneyness_abs'].min()
temp_grouped_df = pd.DataFrame(temp_grouped)
dte_14_42_df = dte_14_42_df[[
    'date_expiry', 'date', 'expiry', 'days_to_expiry', 'moneyness_abs', 'strike', 'implied_volatility', 
    'put_close', 'call_close', 'forward_close', 'close_ul', 'delta', 'gamma', 'theta', 'vega'
]]
dte_14_42_atm_df = temp_grouped_df.merge(dte_14_42_df, on=['date_expiry', 'moneyness_abs'], how='inner')
dte_14_42_atm_df
temp_grouped = dte_14_42_atm_df.groupby('date')['days_to_expiry'].min()
temp_grouped_df = pd.DataFrame(temp_grouped)
dte_14_42_atm_df = temp_grouped_df.merge(dte_14_42_atm_df, on=['date', 'days_to_expiry'], how='inner')

In [ ]:
# Get volatility data
path = '/Users/vamsikrishnasb/My Drive/Financial Analysis/backtesting/vol_estimation_new/NIFTY/2023-04-28.csv'
rvol_df = pd.read_csv(path)
rvol_reduced_df = rvol_df[[
    'date', '20d_c2c_vol'
]]

In [ ]:
# Merge ATM option data with the volatility data
atm_rvol_df = dte_14_42_atm_df.merge(rvol_reduced_df, on=['date'], how='inner')

In [ ]:
# Drop NA from the atm_rvol_df DF
atm_rvol_improved_df = atm_rvol_df.dropna().reset_index().drop('index', axis=1)

In [ ]:
# ATM IV-RV ratio for atm_rvol_improved_df DF
atm_rvol_improved_df['iv_rv_ratio'] = atm_rvol_improved_df['implied_volatility'] / atm_rvol_improved_df['20d_c2c_vol']
atm_rvol_improved_df['vrp'] = atm_rvol_improved_df['iv_rv_ratio'] - 1

In [ ]:
%%time
# Separate out the nearest expiry
temp_df = atm_rvol_improved_df.set_index(['date', 'expiry'])
vrp_month_0_df = temp_df.loc[
    (slice(None) , atm_rvol_improved_df['expiry']), 
    ['days_to_expiry', 'vrp']
].reset_index()
vrp_month_0_df = vrp_month_0_df[
    vrp_month_0_df['date'] >= '2016-01-01'
].reset_index().drop('index', axis=1)

In [ ]:
%%time
# Finds out the 50th percentile VRP value
def iv_rv_ratio_0_mny(ul, dte, rv_type, pc_date):
    path = '/Users/vamsikrishnasb/My Drive/Financial Analysis/backtesting/IVol PC Values/Date Wise IVol Surface/0/'+ul+'/Concatenated/iv_concat.csv'
    df = pd.read_csv(path)
    df = df.sort_values(by=['date'], ascending=[True])
    if 'Unnamed: 0' in df.columns:
        df = df.drop('Unnamed: 0', axis=1)
    df = df.reset_index().drop('index', axis=1)
    iv_df = df[df['date'] <= pc_date][['date', str(dte)]]
    path = '/Users/vamsikrishnasb/My Drive/Financial Analysis/backtesting/vol_estimation_new/'+ul+'/'+vol_date+'.csv'
    df = pd.read_csv(path)
    rv_df = df[[
        'date', 'close_ul', 'log_returns', '20d_c2c_vol'
    ]]
    iv_rv_df = rv_df.merge(iv_df, on=['date'], how='inner')
    iv_rv_df['vrp_20d_c2c'] = iv_rv_df[str(dte)] / iv_rv_df['20d_c2c_vol'] - 1
    vrp_pc.append(iv_rv_df['vrp_20d_c2c'].quantile(0.75, interpolation='nearest'))
    
dates = [
    '2015-12-31', '2016-12-31', '2017-12-31', '2018-12-31', '2019-12-31', '2020-12-31', 
    '2021-12-31', '2022-12-31'
]
dtes = list(range(14, 43))
ul = 'NIFTY'
rv_type = '20d_c2c'
vrp_pc_df = pd.DataFrame(columns=['date'])
for dte in dtes:
    vrp_pc_df[dte] = 0
vrp_pc_df['date'] = dates
vrp_pc_df = vrp_pc_df.set_index('date')
for pc_date in dates:
    vrp_pc = []
    for dte in dtes:
        iv_rv_ratio_0_mny(ul, dte, rv_type, pc_date)
    vrp_pc_df.loc[pc_date] = vrp_pc

In [ ]:
%%time
dtes = vrp_month_0_df['days_to_expiry'].to_list()

c1 = (vrp_month_0_df['date']) <= '2016-12-31'
c2 = (vrp_month_0_df['date']) <= '2017-12-31'
c3 = (vrp_month_0_df['date']) <= '2018-12-31'
c4 = (vrp_month_0_df['date']) <= '2019-12-31'
c5 = (vrp_month_0_df['date']) <= '2020-12-31'
c6 = (vrp_month_0_df['date']) <= '2021-12-31'
c7 = (vrp_month_0_df['date']) <= '2022-12-31'
c8 = (vrp_month_0_df['date']) <= '2023-12-31'

pc_date = [
    '2015-12-31', '2016-12-31', '2017-12-31', '2018-12-31', '2019-12-31', '2020-12-31', 
    '2021-12-31', '2022-12-31'
]
default = 0

vrp_month_0_df['pc_date'] = np.select([c1, c2, c3, c4, c5, c6, c7, c8], pc_date, default=default)
vrp_month_0_df['vrp_50pc_value'] = ''
# for dte, pc_date in zip(dtes, dates):
for dte, i in zip(dtes, range(0, len(vrp_month_0_df))):
    vrp_month_0_df['vrp_50pc_value'][i] = vrp_pc_df.loc[vrp_month_0_df['pc_date'][i]][dte]
vrp_month_0_df = vrp_month_0_df.drop_duplicates(subset=['date'], keep='last')

In [ ]:
# Mark whether a short volatility trade will taken or not, by comparing 'vrp' column
# with 'vrp_50pc_value' column
vrp_month_0_df['trade_decision'] = np.where(vrp_month_0_df['vrp'] >= vrp_month_0_df['vrp_50pc_value'], 1, 0)

In [ ]:
# Merge vrp_month_0_df DF and atm_rvol_improved_df
atm_df = vrp_month_0_df.merge(atm_rvol_improved_df, on=['date', 'expiry', 'days_to_expiry', 'vrp'], how='inner')

In [ ]:
# Multiple operations
# Straddle price, quantities to trade (which is 1 Crore / NIFTY spot), etc.
# atm_df = atm_df.drop(1744) # Dropping a duplicate row
atm_df['straddle_price'] = atm_df['put_close'] + atm_df['call_close']
atm_df['close_ul'] = pd.to_numeric(atm_df['close_ul'], errors='coerce')
atm_df['trade_quantity'] = round(np.floor(10000000.00 / atm_df['close_ul'])/100)*100
atm_df = atm_df.reset_index().drop('index', axis=1)

In [ ]:
# Call delta and put delta
atm_df['call_delta'] = np.where(atm_df['delta'] > 0, atm_df['delta'], 1 + atm_df['delta'])
atm_df['put_delta'] = np.where(atm_df['delta'] < 0, atm_df['delta'], atm_df['delta'] - 1)
atm_df['position_delta'] = -1 * (atm_df['call_delta'] + atm_df['put_delta']) * atm_df['trade_quantity']
atm_df['position_gamma'] = -1 * (atm_df['gamma'] * 2) * atm_df['trade_quantity']
atm_df['position_theta'] = -1 * (atm_df['theta'] * 2) * atm_df['trade_quantity']
atm_df['position_vega'] = -1 * (atm_df['vega'] * 2) * atm_df['trade_quantity']

In [ ]:
# Rename columns in atm_df to make it easier while merging with otm_df
atm_df = atm_df.rename(columns={
    'strike': 'atm_strike',
    'implied_volatility': 'atm_iv', 
    'put_close': 'atm_put_price',
    'call_close': 'atm_call_price',
    'delta': 'atm_delta',
    'gamma': 'atm_gamma',
    'theta': 'atm_theta',
    'vega': 'atm_vega',
    'straddle_price': 'atm_straddle_price',
    'trade_quantity': 'atm_qty',
    'call_delta': 'atm_call_delta',
    'put_delta': 'atm_put_delta',
    'position_delta': 'atm_position_delta',
    'position_gamma': 'atm_position_gamma',
    'position_theta': 'atm_position_theta',
    'position_vega': 'atm_position_vega'
})

In [ ]:
# I should be able to separate out individual trades
# What is a trade? Consecutive rows with trade_decisions column being equal to 1 is a trade!
trade_group_df = pd.DataFrame(columns=['date', 'expiry', 'trade_decision', 'trade_group'])
trade_group_count = 1
expiries = atm_df['expiry'].unique()
for expiry in expiries:
    temp_expiry_df = atm_df[atm_df['expiry'] == expiry][[
        'date', 'expiry', 'trade_decision'
    ]].reset_index().drop('index', axis=1)
    temp_expiry_df['trade_group'] = ''
    for i in range(0, len(temp_expiry_df)):
        if i == 0:
            if (temp_expiry_df['trade_decision'][i] == 1):
                temp_expiry_df['trade_group'][i] = trade_group_count
            else:
                temp_expiry_df['trade_group'][i] = 0
        elif (i > 0) & (i < len(temp_expiry_df)-1):
            if (temp_expiry_df['trade_decision'][i-1] == 1) & (temp_expiry_df['trade_decision'][i] == 1):
                temp_expiry_df['trade_group'][i] = trade_group_count
            elif (temp_expiry_df['trade_decision'][i-1] == 1) & (temp_expiry_df['trade_decision'][i] == 0):
                temp_expiry_df['trade_group'][i] = trade_group_count
                trade_group_count = trade_group_count + 1
            elif (temp_expiry_df['trade_decision'][i-1] == 0) & (temp_expiry_df['trade_decision'][i] == 1):
                temp_expiry_df['trade_group'][i] = trade_group_count
            else:
                temp_expiry_df['trade_group'][i] = 0
        else:
            if (temp_expiry_df['trade_decision'][i-1] == 1):
                temp_expiry_df['trade_group'][i] = trade_group_count
                trade_group_count = trade_group_count + 1
            else:
                temp_expiry_df['trade_group'][i] = 0
    trade_group_count = trade_group_count + 1
    trade_group_df = pd.concat([trade_group_df, temp_expiry_df], axis=0)
    
trade_group_df = trade_group_df.drop('trade_decision', axis=1)
atm_df = atm_df.merge(trade_group_df, on=['date', 'expiry'], how='inner')

In [ ]:
# Define the hedges for the short volatility trade
put_otm_df = df[
    (df['moneyness_close'] <= 0.11)
    & (df['moneyness_close'] >= 0.05)
    & (df['days_to_expiry'].between(14, 42))
    & (df['expiry'].isin(valid_expiry))
    & (df['date'] >= '2016-01-01')
    & ((df['strike'] % 100) == 0)
][[
    'date', 'expiry', 'strike', 'days_to_expiry', 'otm_option_type', 'moneyness_close', 'close_option', 
    'implied_volatility', 'delta', 'gamma', 'theta', 'vega'
]]
put_otm_df['moneyness_abs'] = put_otm_df['moneyness_close'].abs()
temp = put_otm_df.groupby(['date', 'expiry'])['moneyness_abs'].max()
temp_df = pd.DataFrame(temp)
put_otm_lowest_mny_df = put_otm_df.merge(temp_df, on=['date', 'expiry', 'moneyness_abs'], how='inner')
put_otm_lowest_mny_df = put_otm_lowest_mny_df.rename(columns = {
    'strike': 'otm_put_strike',
    'otm_option_type': 'otm_put_option_type',
    'moneyness_close': 'otm_put_moneyness',
    'close_option': 'otm_put_price',
    'implied_volatility': 'otm_put_iv',
    'delta': 'otm_put_delta',
    'gamma': 'otm_put_gamma',
    'theta': 'otm_put_theta',
    'vega' : 'otm_put_vega'
})
put_otm_lowest_mny_df = put_otm_lowest_mny_df[[
    'date', 'expiry', 'days_to_expiry', 'otm_put_strike', 'otm_put_option_type', 'otm_put_moneyness', 
    'otm_put_price', 'otm_put_iv', 'otm_put_delta', 'otm_put_gamma', 'otm_put_theta',
    'otm_put_vega'
]]
temp_grouped = put_otm_lowest_mny_df.groupby('date')['days_to_expiry'].min()
temp_grouped_df = pd.DataFrame(temp_grouped)
put_otm_lowest_mny_df = temp_grouped_df.merge(put_otm_lowest_mny_df, on=['date', 'days_to_expiry'], how='inner')
put_otm_lowest_mny_df = put_otm_lowest_mny_df.drop_duplicates(
    subset=['date'], keep='last', ignore_index=True
)

In [ ]:
call_otm_df = df[
    (df['moneyness_close'] <= -0.01)
    & (df['days_to_expiry'].between(14, 42))
    & (df['expiry'].isin(valid_expiry))
    & (df['date'] >= '2016-01-01')
    & ((df['strike'] % 100) == 0)
][[
    'date', 'expiry', 'strike', 'days_to_expiry', 'otm_option_type', 'moneyness_close', 'close_option', 
    'implied_volatility', 'delta', 'gamma', 'theta', 'vega'
]]
call_otm_df['moneyness_abs'] = call_otm_df['moneyness_close'].abs()
call_otm_df = call_otm_df.merge(put_otm_lowest_mny_df, on=['date', 'expiry', 'days_to_expiry'], how='left')
call_otm_df = call_otm_df[
    (call_otm_df['vega'] <= call_otm_df['otm_put_vega'])
]
temp = call_otm_df.groupby(['date', 'expiry'])['vega'].max()
temp_df = pd.DataFrame(temp)
call_otm_lowest_mny_df  = call_otm_df.merge(temp_df, on=['date', 'expiry', 'vega'], how='inner')
call_otm_lowest_mny_df = call_otm_lowest_mny_df.rename(columns = {
    'strike': 'otm_call_strike',
    'otm_option_type': 'otm_call_option_type',
    'moneyness_close': 'otm_call_moneyness',
    'close_option': 'otm_call_price',
    'implied_volatility': 'otm_call_iv',
    'delta': 'otm_call_delta',
    'gamma': 'otm_call_gamma',
    'theta': 'otm_call_theta',
    'vega' : 'otm_call_vega'
})
call_otm_lowest_mny_df = call_otm_lowest_mny_df[[
    'date', 'expiry', 'days_to_expiry', 'otm_call_strike', 'otm_call_option_type', 'otm_call_moneyness', 
    'otm_call_price', 'otm_call_iv', 'otm_call_delta', 'otm_call_gamma', 'otm_call_theta',
    'otm_call_vega'
]]
temp_grouped = call_otm_lowest_mny_df.groupby('date')['days_to_expiry'].min()
temp_grouped_df = pd.DataFrame(temp_grouped)
call_otm_lowest_mny_df = temp_grouped_df.merge(call_otm_lowest_mny_df, on=['date', 'days_to_expiry'], how='inner')
call_otm_lowest_mny_df = call_otm_lowest_mny_df.drop_duplicates(
    subset=['date'], keep='last', ignore_index=True
)

In [ ]:
# Merge put_otm_df and call_otm_df
otm_df = put_otm_lowest_mny_df.merge(call_otm_lowest_mny_df, on=['date', 'expiry', 'days_to_expiry'], how='inner')

# Merge atm_df with otm_df
atm_otm_df = atm_df.merge(otm_df, on=['date', 'expiry', 'days_to_expiry'], how='inner')

# Filter out wrong values of iv, both atm and otm
atm_otm_df = atm_otm_df[
    (atm_otm_df['atm_iv'].between(0.05, 5))
    & (atm_otm_df['otm_put_iv'].between(0.05, 5))
    & (atm_otm_df['otm_call_iv'].between(0.05, 5))
].reset_index().drop('index', axis=1)


# Calculate the quantities of otm puts and calls
atm_otm_df['otm_put_qty'] = 1
atm_otm_df['otm_call_qty'] = 1
atm_otm_df['otm_put_position_qty_temp'] = np.ceil(atm_otm_df['atm_qty'] * atm_otm_df['otm_put_qty']/100)*100
atm_otm_df['otm_call_position_qty_temp'] = np.ceil(atm_otm_df['atm_qty'] * atm_otm_df['otm_call_qty']/100)*100
atm_otm_df['otm_put_position_qty'] = atm_otm_df[['atm_qty', 'otm_put_position_qty_temp']].max(axis=1)
atm_otm_df['otm_call_position_qty'] = atm_otm_df[['atm_qty', 'otm_call_position_qty_temp']].max(axis=1)

# Calculate over greeks for the position
# Remove unnecessary columns
atm_otm_df['position_delta'] = -1 * (atm_otm_df['atm_call_delta'] + atm_otm_df['atm_put_delta']) * atm_otm_df['atm_qty'] + atm_otm_df['otm_call_delta'] * atm_otm_df['otm_call_position_qty'] / (1) + atm_otm_df['otm_put_delta'] * atm_otm_df['otm_put_position_qty'] / (1)
atm_otm_df['position_gamma'] = -1 * (atm_otm_df['atm_gamma'] * 2) * atm_otm_df['atm_qty'] + atm_otm_df['otm_call_gamma'] * atm_otm_df['otm_call_position_qty'] / (1) + atm_otm_df['otm_put_gamma'] * atm_otm_df['otm_put_position_qty'] / (1)
atm_otm_df['position_theta'] = -1 * (atm_otm_df['atm_theta'] * 2) * atm_otm_df['atm_qty'] + atm_otm_df['otm_call_theta'] * atm_otm_df['otm_call_position_qty'] / (1) + atm_otm_df['otm_put_theta'] * atm_otm_df['otm_put_position_qty'] / (1)
atm_otm_df['position_vega'] = -1 * (atm_otm_df['atm_vega'] * 2) * atm_otm_df['atm_qty'] + atm_otm_df['otm_call_vega'] * atm_otm_df['otm_call_position_qty'] / (1) + atm_otm_df['otm_put_vega'] * atm_otm_df['otm_put_position_qty'] / (1)
atm_otm_improved_df = atm_otm_df[[
    'date', 'expiry', 'days_to_expiry', 'trade_decision', 'trade_group', 'forward_close', 
    'atm_strike',  'atm_put_price', 'atm_call_price', 'close_ul', 'atm_iv', '20d_c2c_vol', 'vrp',
    'atm_delta', 'atm_gamma', 'atm_theta', 'atm_vega', 
    'atm_straddle_price', 'atm_qty', 'atm_call_delta', 'atm_put_delta', 
    'atm_position_delta', 'otm_put_strike', 'otm_put_price', 'otm_put_delta', 
    'otm_put_gamma', 'otm_put_theta', 'otm_put_vega', 'otm_call_strike', 'otm_call_price',
    'otm_call_delta', 'otm_call_gamma', 'otm_call_theta', 'otm_call_vega', 
    'otm_put_qty', 'otm_call_qty', 'otm_put_position_qty', 'otm_call_position_qty',
    'position_delta', 'position_gamma', 'position_theta', 'position_vega'
]].rename(columns={
    '20d_c2c_vol': 'realized_vol',
    'vrp': 'vol_risk_premium'
})

# Calculate position value
atm_otm_improved_df['position_value'] = (- atm_otm_improved_df['atm_straddle_price'] * atm_otm_improved_df['atm_qty']) + (atm_otm_improved_df['otm_put_price'] * atm_otm_improved_df['otm_put_position_qty']) + (atm_otm_improved_df['otm_call_price'] * atm_otm_improved_df['otm_call_position_qty'])

In [ ]:
%%time
trade_groups = list(set(atm_otm_improved_df['trade_group'].unique()) - set([0]))
short_pnl_df = pd.DataFrame(columns=[
    'date', 'options_pnl', 'futures_pnl', 'atm_pnl'
])
delta_adjustments_df = pd.DataFrame(columns=[
    'date', 'expiry', 'atm_strike', 'atm_straddle_price', 'forward_close', 'atm_delta', 
    'atm_qty', 'atm_call_delta', 'atm_put_delta', 'otm_put_strike', 'otm_put_price', 
    'otm_put_delta', 'otm_call_strike', 'otm_call_price', 'otm_call_delta', 
    'position_delta', 'net_delta', 'futures_qty_to_buy_or_sell', 
    'net_futures_qty_post_hedging', 'futures_times_qty'
])
date_list = []
options_pnl_list = []
futures_pnl_list = []
net_pnl_list = []
for tg in trade_groups:
    trade_df = pd.DataFrame(columns=[
        'date', 'expiry', 'atm_strike', 'forward_close', 'atm_delta', 
        'atm_qty', 'otm_put_price', 'otm_put_delta', 'otm_call_price', 'otm_call_delta', 
    ])
    temp_df = atm_otm_improved_df[atm_otm_improved_df['trade_group'] == tg][[
        'date', 'expiry', 'atm_strike', 'atm_straddle_price', 'forward_close', 'atm_delta', 
        'atm_qty', 'otm_put_strike', 'otm_put_price', 'otm_put_delta',
        'otm_call_strike', 'otm_call_price', 'otm_call_delta'
    ]].reset_index().drop('index', axis=1)
    atm_strike = temp_df['atm_strike'].iloc[0]
    otm_put_strike = temp_df['otm_put_strike'].iloc[0]
    otm_call_strike = temp_df['otm_call_strike'].iloc[0]
    expiry = temp_df['expiry'].iloc[0]
    atm_qty = temp_df['atm_qty'].iloc[0]
    trade_df_date = []
    trade_df_expiry = []
    trade_df_atm_strike = []
    trade_df_atm_straddle_price = []
    trade_df_forward_close = []
    trade_df_atm_delta = []
    trade_df_atm_qty = []
    trade_df_otm_put_strike = []
    trade_df_otm_put_price = []
    trade_df_otm_put_delta = []
    trade_df_otm_call_strike = []
    trade_df_otm_call_price = []
    trade_df_otm_call_delta = []
    trade_df_net_delta = []
    trade_df_futures_qty = []
    trade_df_futures_pnl = []
    for i in range(0, len(temp_df)):
        try:
            atm_delta_df = df[
                (df['date'] == temp_df['date'][i])
                & (df['expiry'] == expiry)
                & (df['strike'] == atm_strike)
            ][['date', 'expiry', 'strike', 'forward_close', 'delta', 'put_close', 'call_close']].reset_index().drop('index', axis=1)
            otm_put_delta_df = df[
                (df['date'] == temp_df['date'][i])
                & (df['expiry'] == expiry)
                & (df['strike'] <= otm_put_strike)
            ][['date', 'expiry', 'strike', 'forward_close', 'delta', 'put_close']].reset_index().drop('index', axis=1).sort_values(by=['strike'], ascending=[False])      
            otm_call_delta_df = df[
                (df['date'] == temp_df['date'][i])
                & (df['expiry'] == expiry)
                & (df['strike'] >= otm_call_strike)
            ][['date', 'expiry', 'strike', 'forward_close', 'delta', 'call_close']].reset_index().drop('index', axis=1).sort_values(by=['strike'], ascending=[True])      
            trade_df_date.append(atm_delta_df['date'].iloc[0])
            trade_df_expiry.append(atm_delta_df['expiry'].iloc[0])
            trade_df_atm_strike.append(atm_delta_df['strike'].iloc[0])
            trade_df_forward_close.append(atm_delta_df['forward_close'].iloc[0])
            trade_df_atm_straddle_price.append(atm_delta_df['put_close'].iloc[0] + atm_delta_df['call_close'].iloc[0])
            trade_df_atm_delta.append(atm_delta_df['delta'].iloc[0])
            trade_df_otm_put_strike.append(otm_put_delta_df['strike'].iloc[0])
            trade_df_otm_put_price.append(otm_put_delta_df['put_close'].iloc[0])
            trade_df_otm_put_delta.append(otm_put_delta_df['delta'].iloc[0])
            trade_df_otm_call_strike.append(otm_call_delta_df['strike'].iloc[0])
            trade_df_otm_call_price.append(otm_call_delta_df['call_close'].iloc[0])
            trade_df_otm_call_delta.append(otm_call_delta_df['delta'].iloc[0])
        except:
            atm_delta_df = df[
                (df['date'] == temp_df['date'][i])
                & (df['expiry'] == expiry)
                & (df['strike'].between(atm_strike - 50, atm_strike + 50))
            ][['date', 'expiry', 'strike', 'forward_close', 'delta', 'put_close', 'call_close']].reset_index().drop('index', axis=1)
            otm_put_delta_df = df[
                (df['date'] == temp_df['date'][i])
                & (df['expiry'] == expiry)
                & (df['strike'] <= otm_put_strike)
            ][['date', 'expiry', 'strike', 'forward_close', 'delta', 'put_close']].reset_index().drop('index', axis=1).sort_values(by=['strike'], ascending=[False])      
            otm_call_delta_df = df[
                (df['date'] == temp_df['date'][i])
                & (df['expiry'] == expiry)
                & (df['strike'] >= otm_call_strike)
            ][['date', 'expiry', 'strike', 'forward_close', 'delta', 'call_close']].reset_index().drop('index', axis=1).sort_values(by=['strike'], ascending=[True])      
            trade_df_date.append(atm_delta_df['date'].iloc[0])
            trade_df_expiry.append(atm_delta_df['expiry'].iloc[0])
            trade_df_atm_strike.append(atm_delta_df['strike'].iloc[0])
            trade_df_forward_close.append(atm_delta_df['forward_close'].iloc[0])
            trade_df_atm_straddle_price.append(atm_delta_df['put_close'].iloc[0] + atm_delta_df['call_close'].iloc[0])
            trade_df_atm_delta.append(atm_delta_df['delta'].iloc[0])
            trade_df_otm_put_strike.append(atm_delta_df['strike'].iloc[0])
            trade_df_otm_put_price.append(otm_put_delta_df['put_close'].iloc[0])
            trade_df_otm_put_delta.append(otm_put_delta_df['delta'].iloc[0])
            trade_df_otm_call_strike.append(atm_delta_df['strike'].iloc[0])
            trade_df_otm_call_price.append(otm_call_delta_df['call_close'].iloc[0])
            trade_df_otm_call_delta.append(otm_call_delta_df['delta'].iloc[0])
            continue
    try:
        trade_df['date'] = trade_df_date
        trade_df['expiry'] = trade_df_expiry
        trade_df['atm_strike'] = trade_df_atm_strike
        trade_df['atm_straddle_price'] = trade_df_atm_straddle_price
        trade_df['forward_close'] = trade_df_forward_close
        trade_df['atm_delta'] = trade_df_atm_delta
        trade_df['otm_put_strike'] = trade_df_otm_put_strike
        trade_df['otm_put_price'] = trade_df_otm_put_price
        trade_df['otm_put_delta'] = trade_df_otm_put_delta
        trade_df['otm_call_strike'] = trade_df_otm_call_strike
        trade_df['otm_call_price'] = trade_df_otm_call_price
        trade_df['otm_call_delta'] = trade_df_otm_call_delta
        trade_df['atm_qty'] = atm_qty
        trade_df['atm_call_delta'] = np.where(trade_df['atm_delta'] >= 0, trade_df['atm_delta'], trade_df['atm_delta'] + 1)
        trade_df['atm_put_delta'] = np.where(trade_df['atm_delta'] <= 0, trade_df['atm_delta'], trade_df['atm_delta'] - 1)
        trade_df['otm_put_delta'] = np.where(trade_df['otm_put_delta'] <= 0, trade_df['otm_put_delta'], trade_df['otm_put_delta'] - 1)
        trade_df['otm_call_delta'] = np.where(trade_df['otm_call_delta'] >= 0, trade_df['otm_call_delta'], trade_df['otm_call_delta'] + 1)
        trade_df['position_delta'] = ''
        trade_df['net_delta'] = ''
        trade_df['futures_qty_to_buy_or_sell'] = ''
        trade_df['net_futures_qty_post_hedging'] = ''
        trade_df['futures_times_qty'] = ''
        for i in range(0, len(trade_df)):
            if i == 0:
                trade_df['position_delta'][i] = np.floor((-1*(trade_df['atm_call_delta'][i] + trade_df['atm_put_delta'][i]) + trade_df['otm_call_delta'][i] + trade_df['otm_put_delta'][i]) * atm_qty)
                trade_df['net_delta'][i] = trade_df['position_delta'][i]
                trade_df['futures_qty_to_buy_or_sell'][i] = - trade_df['net_delta'][i]
                trade_df['net_futures_qty_post_hedging'][i] = trade_df['futures_qty_to_buy_or_sell'][i]
            else:
                trade_df['position_delta'][i] = np.floor((-1*(trade_df['atm_call_delta'][i] + trade_df['atm_put_delta'][i]) + trade_df['otm_call_delta'][i] + trade_df['otm_put_delta'][i]) * atm_qty)
                trade_df['net_delta'][i] = trade_df['position_delta'][i] + trade_df['net_futures_qty_post_hedging'][i-1]
                trade_df['futures_qty_to_buy_or_sell'][i] = - trade_df['net_delta'][i]
                trade_df['net_futures_qty_post_hedging'][i] = trade_df['futures_qty_to_buy_or_sell'][i] + trade_df['net_futures_qty_post_hedging'][i-1]
            trade_df['futures_times_qty'][i] = trade_df['forward_close'][i] * trade_df['futures_qty_to_buy_or_sell'][i]    
        atm_pnl = - (trade_df['atm_straddle_price'].iloc[len(trade_df) - 1] - trade_df['atm_straddle_price'].iloc[0]) * trade_df['atm_qty'].iloc[0]
        otm_put_pnl = (trade_df['otm_put_price'].iloc[len(trade_df) - 1] - trade_df['otm_put_price'].iloc[0]) * trade_df['atm_qty'].iloc[0]
        otm_call_pnl = (trade_df['otm_call_price'].iloc[len(trade_df) - 1] - trade_df['otm_call_price'].iloc[0]) * trade_df['atm_qty'].iloc[0]
        options_pnl = atm_pnl + otm_put_pnl + otm_call_pnl
        futures_pnl = trade_df['futures_times_qty'].sum() + (-1 * trade_df['net_futures_qty_post_hedging'].iloc[len(trade_df) - 1] * trade_df['forward_close'].iloc[len(trade_df) - 1])
        net_pnl = futures_pnl + options_pnl
        date_list.append(trade_df['date'].iloc[len(trade_df)-1])
        options_pnl_list.append(options_pnl)
        futures_pnl_list.append(futures_pnl)
        net_pnl_list.append(net_pnl)
        delta_adjustments_df = pd.concat([delta_adjustments_df, trade_df], axis=0)
    except:
        print("B", tg)
        continue
delta_adjustments_df = delta_adjustments_df.reset_index().drop('index', axis=1)
short_pnl_df['date'] = date_list    
short_pnl_df['options_pnl'] = options_pnl_list
short_pnl_df['futures_pnl'] = futures_pnl_list
short_pnl_df['atm_pnl'] = net_pnl_list
short_pnl_df['cumulative_atm_pnl'] = short_pnl_df['atm_pnl'].cumsum()

# Make 'short_pnl_df' compatible with other pnl_df's
short_pnl_df['date'] = pd.to_datetime(short_pnl_df['date'], format='mixed')
short_pnl_df['month'] = short_pnl_df['date'].apply(lambda x: "{}-{}".format(x.month_name()[:3],x.year))
monthly_atm_pnl = short_pnl_df.groupby(['month'], sort=False)['atm_pnl'].sum()
monthly_short_pnl_df = pd.DataFrame(monthly_atm_pnl)
monthly_short_pnl_df = monthly_short_pnl_df.reset_index()
monthly_short_pnl_df['atm_pnl'] = monthly_short_pnl_df['atm_pnl'].fillna(0)
monthly_short_pnl_df['cumulative_atm_pnl'] = monthly_short_pnl_df['atm_pnl'].cumsum()

In [ ]:
# entry_exit_df
entry_df = pd.DataFrame(atm_otm_improved_df.groupby('trade_group').first()).reset_index()
entry_df = entry_df[
    (entry_df['trade_group'] != 0)
].reset_index().drop('index', axis=1)[[
    'trade_group', 'date', 'atm_strike', 'otm_put_strike', 'atm_iv', 'realized_vol',
    'vol_risk_premium', 'otm_call_strike', 'position_delta', 'position_gamma', 
    'position_theta', 'position_vega'
]].rename(columns={
    'date': 'entry_date'
})
exit_df = pd.DataFrame(atm_otm_improved_df.groupby('trade_group').last()).reset_index()
exit_df = exit_df[
    (exit_df['trade_group'] != 0)
].reset_index().drop('index', axis=1)[[
    'trade_group', 'date'
]].rename(columns={
    'date': 'exit_date'
})
entry_exit_short_df = entry_df.merge(exit_df, on=['trade_group'], how='inner')
entry_exit_short_df = entry_exit_short_df.rename(columns={
    'trade_group': 'trade_id'
})[[
    'trade_id', 'entry_date', 'exit_date', 'atm_iv', 'realized_vol',
    'vol_risk_premium', 'atm_strike', 'otm_put_strike', 'otm_call_strike', 
    'position_delta', 'position_gamma', 'position_theta', 'position_vega'
]]

def convert_to_percentage(x):
    return "{:.2%}".format(x)

entry_exit_short_df['atm_iv'] = entry_exit_short_df['atm_iv'].apply(convert_to_percentage)
entry_exit_short_df['realized_vol'] = entry_exit_short_df['realized_vol'].apply(convert_to_percentage)
entry_exit_short_df['vol_risk_premium'] = entry_exit_short_df['vol_risk_premium'].apply(convert_to_percentage)

In [ ]:
entry_exit_short_df

In [ ]:
delta_adjustments_df = pd.DataFrame(delta_adjustments_df.merge(atm_otm_improved_df[['date', 'expiry', 'trade_group']], on=['date', 'expiry'], how='left'))
delta_adjustments_df = delta_adjustments_df[[
    'trade_group', 'date', 'expiry', 'atm_strike', 'atm_straddle_price', 'forward_close',
    'atm_delta', 'atm_qty', 'atm_call_delta', 'atm_put_delta', 'otm_put_strike',
    'otm_put_price', 'otm_put_delta', 'otm_call_strike', 'otm_call_price', 'otm_call_delta',
    'position_delta', 'net_delta', 'futures_qty_to_buy_or_sell',
    'net_futures_qty_post_hedging'
]].rename(columns={
    'trade_group': 'trade_id'
})

In [ ]:
delta_adjustments_df

In [ ]:
# Systematic deep otm trade
put_deep_otm_df = df[
    (df['moneyness_close'] >= 0.15)
    & (df['days_to_expiry'].between(14, 42))
    & (df['expiry'].isin(valid_expiry))
    & (df['date'] >= '2016-01-01')
    & ((df['strike'] % 100) == 0)
][[
    'date', 'expiry', 'strike', 'days_to_expiry', 'otm_option_type', 'moneyness_close', 'close_option', 
    'implied_volatility', 'delta', 'gamma', 'theta', 'vega'
]]
put_deep_otm_df['moneyness_abs'] = put_deep_otm_df['moneyness_close'].abs()
temp = put_deep_otm_df.groupby(['date', 'expiry'])['moneyness_abs'].min()
temp_df = pd.DataFrame(temp)
put_deep_otm_lowest_mny_df = put_deep_otm_df.merge(temp_df, on=['date', 'expiry', 'moneyness_abs'], how='inner')
put_deep_otm_lowest_mny_df = put_deep_otm_lowest_mny_df.rename(columns = {
    'strike': 'otm_put_strike',
    'otm_option_type': 'otm_put_option_type',
    'moneyness_close': 'otm_put_moneyness',
    'close_option': 'otm_put_price',
    'implied_volatility': 'otm_put_iv',
    'delta': 'otm_put_delta',
    'gamma': 'otm_put_gamma',
    'theta': 'otm_put_theta',
    'vega' : 'otm_put_vega'
})
put_deep_otm_lowest_mny_df = put_deep_otm_lowest_mny_df[[
    'date', 'expiry', 'days_to_expiry', 'otm_put_strike', 'otm_put_option_type', 'otm_put_moneyness', 
    'otm_put_price', 'otm_put_iv', 'otm_put_delta', 'otm_put_gamma', 'otm_put_theta',
    'otm_put_vega'
]]
temp_grouped = put_deep_otm_lowest_mny_df.groupby('date')['days_to_expiry'].min()
temp_grouped_df = pd.DataFrame(temp_grouped)
put_deep_otm_lowest_mny_df = temp_grouped_df.merge(put_deep_otm_lowest_mny_df, on=['date', 'days_to_expiry'], how='inner')
put_deep_otm_lowest_mny_df = put_deep_otm_lowest_mny_df.drop_duplicates(
    subset=['date'], keep='last', ignore_index=True
)

In [ ]:
# Define deep_otm_df
# Naively, define deep_otm_df as the following:
# For call deep_otm options: moneyness >= 0.1
# For call deep_otm options: moneyness <= -0.05
call_deep_otm_df = df[
    (df['moneyness_close'] <= -0.1)
    & (df['days_to_expiry'].between(14, 42))
    & (df['expiry'].isin(valid_expiry))
    & (df['date'] >= '2016-01-01')
    & ((df['strike'] % 100) == 0)
][[
    'date', 'expiry', 'strike', 'days_to_expiry', 'otm_option_type', 'moneyness_close', 'close_option', 
    'implied_volatility', 'delta', 'gamma', 'theta', 'vega'
]]
call_deep_otm_df['moneyness_abs'] = call_deep_otm_df['moneyness_close'].abs()
temp = call_deep_otm_df.groupby(['date', 'expiry'])['moneyness_abs'].min()
temp_df = pd.DataFrame(temp)
call_deep_otm_lowest_mny_df = call_deep_otm_df.merge(temp_df, on=['date', 'expiry', 'moneyness_abs'], how='inner')
call_deep_otm_lowest_mny_df = call_deep_otm_lowest_mny_df.rename(columns = {
    'strike': 'otm_call_strike',
    'otm_option_type': 'otm_call_option_type',
    'moneyness_close': 'otm_call_moneyness',
    'close_option': 'otm_call_price',
    'implied_volatility': 'otm_call_iv',
    'delta': 'otm_call_delta',
    'gamma': 'otm_call_gamma',
    'theta': 'otm_call_theta',
    'vega' : 'otm_call_vega'
})
call_deep_otm_lowest_mny_df = call_deep_otm_lowest_mny_df[[
    'date', 'expiry', 'days_to_expiry', 'otm_call_strike', 'otm_call_option_type', 'otm_call_moneyness', 
    'otm_call_price', 'otm_call_iv', 'otm_call_delta', 'otm_call_gamma', 'otm_call_theta',
    'otm_call_vega'
]]
temp_grouped = call_deep_otm_lowest_mny_df.groupby('date')['days_to_expiry'].min()
temp_grouped_df = pd.DataFrame(temp_grouped)
call_deep_otm_lowest_mny_df = temp_grouped_df.merge(call_deep_otm_lowest_mny_df, on=['date', 'days_to_expiry'], how='inner')
call_deep_otm_lowest_mny_df = call_deep_otm_lowest_mny_df.drop_duplicates(
    subset=['date'], keep='last', ignore_index=True
)

In [ ]:
# Merge put_deep_otm_df and call_deep_otm_df
deep_otm_df = put_deep_otm_lowest_mny_df.merge(call_deep_otm_lowest_mny_df, on=['date', 'expiry', 'days_to_expiry'], how='inner')

# Merge atm_df with otm_df
atm_deep_otm_df = atm_df.merge(deep_otm_df, on=['date', 'expiry', 'days_to_expiry'], how='inner')

# Filter out wrong values of iv, both atm and otm
atm_deep_otm_df = atm_deep_otm_df[
    (atm_deep_otm_df['atm_iv'].between(0.05, 5))
    & (atm_deep_otm_df['otm_put_iv'].between(0.05, 5))
    & (atm_deep_otm_df['otm_call_iv'].between(0.05, 5))
].reset_index().drop('index', axis=1)


# Calculate the quantities of otm puts and calls to be found
atm_deep_otm_df['otm_put_qty'] = atm_deep_otm_df['atm_vega'] / atm_deep_otm_df['otm_put_vega'] / 2
atm_deep_otm_df['otm_call_qty'] = atm_deep_otm_df['atm_vega'] / atm_deep_otm_df['otm_call_vega'] / 4
atm_deep_otm_df['otm_put_position_qty_temp'] = np.ceil(atm_deep_otm_df['atm_qty'] * atm_deep_otm_df['otm_put_qty']/100)*100
atm_deep_otm_df['otm_call_position_qty_temp'] = np.ceil(atm_deep_otm_df['atm_qty'] * atm_deep_otm_df['otm_call_qty']/100)*100
atm_deep_otm_df['otm_put_position_qty'] = atm_deep_otm_df[['atm_qty', 'otm_put_position_qty_temp']].max(axis=1)
atm_deep_otm_df['otm_call_position_qty'] = atm_deep_otm_df[['atm_qty', 'otm_call_position_qty_temp']].max(axis=1)

# Calculate over greeks for the position
# Remove unnecessary columns
atm_deep_otm_df['position_delta'] = -1 * (atm_deep_otm_df['atm_call_delta'] + atm_deep_otm_df['atm_put_delta']) * atm_deep_otm_df['atm_qty'] + atm_deep_otm_df['otm_call_delta'] * atm_deep_otm_df['otm_call_position_qty'] / (1) + atm_deep_otm_df['otm_put_delta'] * atm_deep_otm_df['otm_put_position_qty'] / (1)
atm_deep_otm_df['position_gamma'] = -1 * (atm_deep_otm_df['atm_gamma'] * 2) * atm_deep_otm_df['atm_qty'] + atm_deep_otm_df['otm_call_gamma'] * atm_deep_otm_df['otm_call_position_qty'] / (1) + atm_deep_otm_df['otm_put_gamma'] * atm_deep_otm_df['otm_put_position_qty'] / (1)
atm_deep_otm_df['position_theta'] = -1 * (atm_deep_otm_df['atm_theta'] * 2) * atm_deep_otm_df['atm_qty'] + atm_deep_otm_df['otm_call_theta'] * atm_deep_otm_df['otm_call_position_qty'] / (1) + atm_deep_otm_df['otm_put_theta'] * atm_deep_otm_df['otm_put_position_qty'] / (1)
atm_deep_otm_df['position_vega'] = -1 * (atm_deep_otm_df['atm_vega'] * 2) * atm_deep_otm_df['atm_qty'] + atm_deep_otm_df['otm_call_vega'] * atm_deep_otm_df['otm_call_position_qty'] / (1) + atm_deep_otm_df['otm_put_vega'] * atm_deep_otm_df['otm_put_position_qty'] / (1)
atm_deep_otm_improved_df = atm_deep_otm_df[[
    'date', 'expiry', 'days_to_expiry', 'trade_decision', 'trade_group', 'atm_strike', 
    'atm_put_price', 'atm_call_price', 'close_ul', 'atm_delta', 'atm_gamma', 'atm_theta', 
    'atm_vega', 'atm_straddle_price', 'atm_qty', 'atm_call_delta', 'atm_put_delta', 
    'atm_position_delta', 'otm_put_strike', 'otm_put_price', 'otm_put_delta', 
    'otm_put_gamma', 'otm_put_theta', 'otm_put_vega', 'otm_call_strike', 'otm_call_price',
    'otm_call_delta', 'otm_call_gamma', 'otm_call_theta', 'otm_call_vega', 
    'otm_put_qty', 'otm_call_qty', 'otm_put_position_qty', 'otm_call_position_qty',
    'position_delta', 'position_gamma', 'position_theta', 'position_vega'
]]

# Calculate position value
atm_deep_otm_improved_df['position_value'] = (- atm_deep_otm_improved_df['atm_straddle_price'] * atm_deep_otm_improved_df['atm_qty']) + (atm_deep_otm_improved_df['otm_put_price'] * atm_deep_otm_improved_df['otm_put_position_qty']) + (atm_deep_otm_improved_df['otm_call_price'] * atm_deep_otm_improved_df['otm_call_position_qty'])

In [ ]:
short_greeks_df = delta_adjustments_df[[
    'date', 'expiry', 'atm_strike', 'otm_put_strike', 'otm_call_strike', 'atm_qty', 'position_delta'
]]
atm_greeks = df[['date', 'expiry', 'strike', 'gamma', 'theta', 'vega']].rename(columns={
    'strike': 'atm_strike',
    'gamma': 'atm_gamma',
    'theta': 'atm_theta',
    'vega': 'atm_vega'
})
otm_put_greeks = df[['date', 'expiry', 'strike', 'gamma', 'theta', 'vega']].rename(columns={
    'strike': 'otm_put_strike',
    'gamma': 'otm_put_gamma',
    'theta': 'otm_put_theta',
    'vega': 'otm_put_vega'
})
otm_call_greeks = df[['date', 'expiry', 'strike', 'gamma', 'theta', 'vega']].rename(columns={
    'strike': 'otm_call_strike',
    'gamma': 'otm_call_gamma',
    'theta': 'otm_call_theta',
    'vega': 'otm_call_vega'
})
short_greeks_df = pd.DataFrame(short_greeks_df.merge(atm_greeks, on=['date', 'expiry', 'atm_strike']))
short_greeks_df = pd.DataFrame(short_greeks_df.merge(otm_put_greeks, on=['date', 'expiry', 'otm_put_strike']))
short_greeks_df = pd.DataFrame(short_greeks_df.merge(otm_call_greeks, on=['date', 'expiry', 'otm_call_strike']))
short_greeks_df['position_gamma'] = (- short_greeks_df['atm_gamma'] * 2 + short_greeks_df['otm_put_gamma'] + short_greeks_df['otm_call_gamma']) * short_greeks_df['atm_qty']
short_greeks_df['position_theta'] = (- short_greeks_df['atm_theta'] * 2 + short_greeks_df['otm_put_theta'] + short_greeks_df['otm_call_theta']) * short_greeks_df['atm_qty']     
short_greeks_df['position_vega'] = (- short_greeks_df['atm_vega'] * 2 + short_greeks_df['otm_put_vega'] + short_greeks_df['otm_call_vega']) * short_greeks_df['atm_qty']         
short_greeks_df = short_greeks_df[[
    'date', 'expiry', 'atm_strike', 'otm_put_strike', 'otm_call_strike', 
    'position_delta', 'position_gamma', 'position_theta', 'position_vega'
]]
short_greeks_df

In [ ]:
# pd.DataFrame(deep_otm_df.groupby('expiry').first()).reset_index()
long_df = atm_deep_otm_improved_df[[
    'date', 'expiry', 'otm_put_strike', 'otm_call_strike', 
    'otm_put_delta', 'otm_put_gamma', 'otm_put_theta', 'otm_put_vega', 'otm_put_position_qty',
    'otm_call_delta', 'otm_call_gamma', 'otm_call_theta', 'otm_call_vega', 
    'otm_call_position_qty'
]].rename(columns={
    'otm_put_strike': 'otm_deep_put_strike',
    'otm_call_strike': 'otm_deep_call_strike'
})
short_df = short_greeks_df.rename(columns={
    'position_delta': 'short_delta', 
    'position_gamma': 'short_gamma', 
    'position_theta': 'short_theta', 
    'position_vega': 'short_vega', 
})
long_short_df = pd.DataFrame(long_df.merge(short_df, on=['expiry', 'date'], how='left')).reset_index()
long_short_df['short_delta'] = long_short_df['short_delta'].fillna(0)
long_short_df['short_gamma'] = long_short_df['short_gamma'].fillna(0)
long_short_df['short_theta'] = long_short_df['short_theta'].fillna(0)
long_short_df['short_vega'] = long_short_df['short_vega'].fillna(0)
long_short_df['position_delta'] = long_short_df['short_delta'] + long_short_df['otm_put_delta'] * long_short_df['otm_put_position_qty'] + long_short_df['otm_call_delta'] * long_short_df['otm_call_position_qty']
long_short_df['position_gamma'] = long_short_df['short_gamma'] + long_short_df['otm_put_gamma'] * long_short_df['otm_put_position_qty'] + long_short_df['otm_call_gamma'] * long_short_df['otm_call_position_qty']
long_short_df['position_theta'] = long_short_df['short_theta'] + long_short_df['otm_put_theta'] * long_short_df['otm_put_position_qty'] + long_short_df['otm_call_theta'] * long_short_df['otm_call_position_qty']
long_short_df['position_vega'] = long_short_df['short_vega'] + long_short_df['otm_put_vega'] * long_short_df['otm_put_position_qty'] + long_short_df['otm_call_vega'] * long_short_df['otm_call_position_qty']
long_short_df = long_short_df.rename(columns={
    'expiry': 'exit_date',
    'date': 'entry_date'
})[[
    'entry_date', 'exit_date', 'otm_deep_put_strike', 'otm_deep_call_strike', 
    'atm_strike', 'otm_put_strike', 'otm_call_strike', 
    'position_delta', 'position_gamma', 'position_theta', 'position_vega'
]]
long_short_df

In [ ]:
%%time
# "Naive" deep otm strategy
# Probably, the only non-naive part about it is the care given to the cheapness or costliness 
# of the option. But like always, so much room for improvement.
expiries = list(atm_deep_otm_improved_df['expiry'].unique())
deep_otm_date = []
deep_otm_expiry = []
deep_otm_dte = []
deep_otm_put_strike = []
deep_otm_put_position_qty = []
deep_otm_put_price = []
deep_otm_put_trade_df = pd.DataFrame(columns=[
    'date', 'expiry', 'days_to_expiry', 'otm_put_strike', 'otm_put_position_qty', 
    'otm_put_price'
])
for expiry in expiries:
    temp_deep_otm_df = pd.DataFrame()
    temp_deep_otm_df = atm_deep_otm_improved_df[
        atm_deep_otm_improved_df['expiry'] == expiry
    ][[
        'date', 'expiry', 'days_to_expiry', 'otm_put_strike', 'otm_put_position_qty', 
        'otm_put_price'
    ]].reset_index().drop('index', axis=1)
    deep_otm_date.append(temp_deep_otm_df['date'].iloc[0])
    deep_otm_expiry.append(temp_deep_otm_df['expiry'].iloc[0])
    deep_otm_dte.append(temp_deep_otm_df['days_to_expiry'].iloc[0])
    deep_otm_put_strike.append(temp_deep_otm_df['otm_put_strike'].iloc[0])
    deep_otm_put_position_qty.append(temp_deep_otm_df['otm_put_position_qty'].iloc[0])
    deep_otm_put_price.append(temp_deep_otm_df['otm_put_price'].iloc[0])
deep_otm_put_trade_df['date'] = deep_otm_date
deep_otm_put_trade_df['expiry'] = deep_otm_expiry
deep_otm_put_trade_df['days_to_expiry'] = deep_otm_dte
deep_otm_put_trade_df['otm_put_strike'] = deep_otm_put_strike
deep_otm_put_trade_df['otm_put_position_qty'] = deep_otm_put_position_qty
deep_otm_put_trade_df['otm_put_price'] = deep_otm_put_price
square_off_df = df[
    df['days_to_expiry'] == 0
][['expiry', 'strike', 'put_close']].rename(
    columns={
        'strike': 'otm_put_strike',
        'put_close': 'otm_put_price_end'
    }
)
# print(square_off_df.dtypes)
deep_otm_put_trade_df = deep_otm_put_trade_df.merge(square_off_df, on=['expiry', 'otm_put_strike'], how='inner')
deep_otm_put_trade_df['deep_otm_put_pnl'] = (deep_otm_put_trade_df['otm_put_price_end'] - deep_otm_put_trade_df['otm_put_price']) * deep_otm_put_trade_df['otm_put_position_qty']
deep_otm_put_pnl_df = deep_otm_put_trade_df[['expiry', 'deep_otm_put_pnl']]

# Make 'deep_otm_put_pnl_df' compatible with other pnl_df's
deep_otm_put_pnl_df['expiry'] = pd.to_datetime(deep_otm_put_pnl_df['expiry'], format='mixed')
deep_otm_put_pnl_df['month'] = deep_otm_put_pnl_df['expiry'].apply(lambda x: "{}-{}".format(x.month_name()[:3],x.year))
monthly_deep_otm_put_pnl = deep_otm_put_pnl_df.groupby(['month'], sort=False)['deep_otm_put_pnl'].sum()
monthly_deep_otm_put_pnl_df = pd.DataFrame(monthly_deep_otm_put_pnl)
monthly_deep_otm_put_pnl_df = monthly_deep_otm_put_pnl_df.reset_index()
monthly_deep_otm_put_pnl_df['deep_otm_put_pnl'] = monthly_deep_otm_put_pnl_df['deep_otm_put_pnl'].fillna(0)
monthly_deep_otm_put_pnl_df['cumulative_deep_otm_put_pnl'] = monthly_deep_otm_put_pnl_df['deep_otm_put_pnl'].cumsum()

In [ ]:
%%time
# "Naive" deep otm strategy
# Probably, the only non-naive part about it is the care given to the cheapness or costliness 
# of the option. But like always, so much room for improvement.
expiries = list(atm_deep_otm_improved_df['expiry'].unique())
deep_otm_date = []
deep_otm_expiry = []
deep_otm_dte = []
deep_otm_call_strike = []
deep_otm_call_position_qty = []
deep_otm_call_price = []
deep_otm_call_trade_df = pd.DataFrame(columns=[
    'date', 'expiry', 'days_to_expiry', 'otm_call_strike', 'otm_call_position_qty', 
    'otm_call_price'
])
for expiry in expiries:
    temp_deep_otm_df = pd.DataFrame()
    temp_deep_otm_df = atm_deep_otm_improved_df[
        atm_deep_otm_improved_df['expiry'] == expiry
    ][[
        'date', 'expiry', 'days_to_expiry', 'otm_call_strike', 'otm_call_position_qty', 
        'otm_call_price'
    ]].reset_index().drop('index', axis=1)
    deep_otm_date.append(temp_deep_otm_df['date'].iloc[0])
    deep_otm_expiry.append(temp_deep_otm_df['expiry'].iloc[0])
    deep_otm_dte.append(temp_deep_otm_df['days_to_expiry'].iloc[0])
    deep_otm_call_strike.append(temp_deep_otm_df['otm_call_strike'].iloc[0])
    deep_otm_call_position_qty.append(temp_deep_otm_df['otm_call_position_qty'].iloc[0])
    deep_otm_call_price.append(temp_deep_otm_df['otm_call_price'].iloc[0])
deep_otm_call_trade_df['date'] = deep_otm_date
deep_otm_call_trade_df['expiry'] = deep_otm_expiry
deep_otm_call_trade_df['days_to_expiry'] = deep_otm_dte
deep_otm_call_trade_df['otm_call_strike'] = deep_otm_call_strike
deep_otm_call_trade_df['otm_call_position_qty'] = deep_otm_call_position_qty
deep_otm_call_trade_df['otm_call_price'] = deep_otm_call_price
square_off_df = df[
    df['days_to_expiry'] == 0
][['expiry', 'strike', 'call_close']].rename(
    columns={
        'strike': 'otm_call_strike',
        'call_close': 'otm_call_price_end'
    }
)
# print(square_off_df.dtypes)
deep_otm_call_trade_df = deep_otm_call_trade_df.merge(square_off_df, on=['expiry', 'otm_call_strike'], how='inner')
deep_otm_call_trade_df['deep_otm_call_pnl'] = (deep_otm_call_trade_df['otm_call_price_end'] - deep_otm_call_trade_df['otm_call_price']) * deep_otm_call_trade_df['otm_call_position_qty']
deep_otm_call_pnl_df = deep_otm_call_trade_df[['expiry', 'deep_otm_call_pnl']]

# Make 'deep_otm_call_pnl_df' compatible with other pnl_df's
deep_otm_call_pnl_df['expiry'] = pd.to_datetime(deep_otm_call_pnl_df['expiry'], format='mixed')
deep_otm_call_pnl_df['month'] = deep_otm_call_pnl_df['expiry'].apply(lambda x: "{}-{}".format(x.month_name()[:3],x.year))
monthly_deep_otm_call_pnl = deep_otm_call_pnl_df.groupby(['month'], sort=False)['deep_otm_call_pnl'].sum()
monthly_deep_otm_call_pnl_df = pd.DataFrame(monthly_deep_otm_call_pnl)
monthly_deep_otm_call_pnl_df = monthly_deep_otm_call_pnl_df.reset_index()
monthly_deep_otm_call_pnl_df['deep_otm_call_pnl'] = monthly_deep_otm_call_pnl_df['deep_otm_call_pnl'].fillna(0)
monthly_deep_otm_call_pnl_df['cumulative_deep_otm_call_pnl'] = monthly_deep_otm_call_pnl_df['deep_otm_call_pnl'].cumsum()

In [ ]:
pnl_df_01 = monthly_deep_otm_put_pnl_df.merge(monthly_deep_otm_call_pnl_df, on=['month'], how='left', sort=False)
pnl_df = pnl_df_01.merge(monthly_short_pnl_df, on=['month'], how='left', sort=False)
pnl_df['atm_pnl'] = pnl_df['atm_pnl'].fillna(0)
pnl_df['deep_otm_put_pnl'] = pnl_df['deep_otm_put_pnl'].fillna(0)
pnl_df['deep_otm_call_pnl'] = pnl_df['deep_otm_call_pnl'].fillna(0)
pnl_df['cumulative_atm_pnl'] = pnl_df['atm_pnl'].cumsum()
pnl_df['cumulative_deep_otm_put_pnl'] = pnl_df['deep_otm_put_pnl'].cumsum()
pnl_df['cumulative_deep_otm_call_pnl'] = pnl_df['deep_otm_call_pnl'].cumsum()
pnl_df['pnl'] = pnl_df['deep_otm_put_pnl'] + pnl_df['deep_otm_call_pnl'] + pnl_df['atm_pnl']
pnl_df['cumulative_pnl'] = pnl_df['pnl'].cumsum()
pnl_df['short_pnl'] = pnl_df['atm_pnl']
pnl_df['long_pnl'] = pnl_df['deep_otm_put_pnl'] + pnl_df['deep_otm_call_pnl']
pnl_df['cumulative_short_pnl'] = pnl_df['short_pnl'].cumsum()
pnl_df['cumulative_long_pnl'] = pnl_df['long_pnl'].cumsum()
pnl_df = pnl_df[['month', 'atm_pnl', 'deep_otm_put_pnl', 'deep_otm_call_pnl', 'cumulative_atm_pnl', 'cumulative_deep_otm_put_pnl', 'cumulative_deep_otm_call_pnl', 'short_pnl', 'cumulative_short_pnl', 'long_pnl', 'cumulative_long_pnl', 'pnl', 'cumulative_pnl']]

In [ ]:
pnl_df

In [ ]:
plot_df = pd.melt(pnl_df, id_vars=['month'], value_vars=['cumulative_short_pnl', 'cumulative_long_pnl', 'cumulative_pnl'])
fig = px.line(plot_df, x='month', y='value', color='variable', log_y=False, title="PnL")
fig.show()